In [1]:
import numpy as np
import pandas as pd
import re
import string
# import matplotlib.pyplot as plt

# the Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
# function to split the data for cross-validation
from sklearn.model_selection import train_test_split
# function for transforming documents into counts
from sklearn.feature_extraction.text import CountVectorizer
# function for encoding categories
from sklearn.preprocessing import LabelEncoder

#### Create a list for column headers since the default dataset does not have header names

In [2]:
column_names= ['ID','TITLE','URL','PUBLISHER','CATEGORY','STORY','HOSTNAME','TIMESTAMP']

## Please use the appropriate seperator or delimiter. Very Tricky !

In [3]:
news_df = pd.read_csv('newsCorpora.csv',sep='	',names=column_names)


In [4]:
import pyes # For documentation around pyes.es : https://pyes.readthedocs.org/en/latest/references/pyes.es.html
import json

In [9]:
X_independent = pd.DataFrame(news_df.iloc[1:5,:2].values)

In [12]:
type(X_independent)

pandas.core.frame.DataFrame

In [13]:
X_independent

,0,1
0,2,Fed's Charles Plosser sees high bar for change...
1,3,US open: Stocks fall after Fed official hints ...
2,4,"Fed risks falling 'behind the curve', Charles ..."
3,5,Fed's Plosser: Nasty Weather Has Curbed Job Gr...


In [14]:
# Add a id for looping into elastic search index
X_independent["no_index"] = [x+1 for x in range(len(X_independent[0]))]

In [15]:
# Convert into json
temp_newsdf = X_independent.to_json(orient = "records")

In [16]:
# Load each record into json format before bulk
newsdf_json= json.loads(temp_newsdf)
print(newsdf_json[0])

{'0': 2, '1': "Fed's Charles Plosser sees high bar for change in pace of tapering", 'no_index': 1}


In [17]:
# Create an index into elastic search. If we need to create a mapping, we ought to do it here
index_name = 'python_to_elastic'
type_name = 'pyelastic'
es= pyes.ES()

In [18]:
i=1
for doc in newsdf_json:
    if(i<5):
        print(doc)
        i=i+1

{'0': 2, '1': "Fed's Charles Plosser sees high bar for change in pace of tapering", 'no_index': 1}
{'0': 3, '1': 'US open: Stocks fall after Fed official hints at accelerated tapering', 'no_index': 2}
{'0': 4, '1': "Fed risks falling 'behind the curve', Charles Plosser says", 'no_index': 3}
{'0': 5, '1': "Fed's Plosser: Nasty Weather Has Curbed Job Growth", 'no_index': 4}


In [21]:
for doc in newsdf_json:
    es.index(doc, index_name, type_name, id=doc['no_index'])

#### Use the parameter 'names' in read_csv() to add column_names to the dataset.

In [ ]:
X_independent = news_df.iloc[:,:2].values

In [ ]:
X_independent

In [ ]:
news_df.head(1) # Displaying just one row

In [ ]:
news_df.count()
len(news_df)

In [ ]:
headline = list(news_df["TITLE"])

In [ ]:
headline1 = []
for elem in headline:
    s = elem.lower()
    trans = str.maketrans("","",string.punctuation)
    s = s.translate(trans)
    headline1.append(s)
headline1[:10]

In [ ]:
news_df["FORM_HEADLINE"] = headline1

In [ ]:
X_independent = news_df[["ID","FORM_HEADLINE"]]

In [ ]:
X_independent

In [ ]:
Y_dependent = news_df["CATEGORY"]

In [ ]:
Y_dependent

In [ ]:
news_df.head()

In [ ]:
encoder = LabelEncoder()
Y_dependent = encoder.fit_transform(Y_dependent)
Y_dependent

In [ ]:
vec = CountVectorizer()
x = vec.fit_transform(news_df["FORM_HEADLINE"])
x

In [ ]:
x.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, Y_dependent, test_size=0.2)

In [ ]:
x_train.shape

In [ ]:
x_train

In [ ]:
nb = MultinomialNB()
nb.fit(x_train, y_train)

In [ ]:
nb.score(x_test,y_test)

In [ ]:
x1 = x_test[:3,:]

In [ ]:
x1

In [ ]:
x1.shape

In [ ]:
x1=list(x1)

In [ ]:
x1

In [ ]:
h = list(news_df.iloc[1000:1003,8])

In [ ]:
news_df.iloc[1000:1003,8]


In [ ]:
type(h)

In [ ]:
type(h)
h

In [ ]:
news_df.iloc[1000:1003,4]

In [ ]:
test1 = vec.transform(h)

In [ ]:
test1

In [ ]:
y = nb.predict(test1)
y

In [ ]:
encoder.inverse_transform(Y_dependent)

In [ ]:
a = [0,1,2,3]

In [ ]:
encoder.inverse_transform(a)

In [ ]:
%store news_df